<a href="https://colab.research.google.com/github/egioia92/NanoGPT/blob/main/NanoGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The base components of GPT comes from a paper called "Attetion is all you need" that proposed the Transormer architecture. GPT stands for Generatively Pretrained Transformer, so transformer is the neural network that does all the heavy work under the wood.

The transformer architecture has been copy pasted with small modification in a huge number of application in AI and that includes at the core chatgpt.

Let's build a transformer based language model in particular a character level language model trained on all Shakespare


In [ ]:
#We always start with a dataset to train on
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-04-07 15:39:18--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.03s   

2024-04-07 15:39:18 (35.5 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
#We read it in to inspect it
with open ('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [ ]:
print("length of dataset in characters:", len(text))

length of dataset in characters: 1115394


In [ ]:
#Let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [ ]:
#Let's see all the unique characters that occur in the text
#Sets are an unordered collection of unique elements
#These are the possible characters that the model can see or emit
chars = sorted(set(text))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [ ]:
#We want to develop some strategy to tokenize the input text
#Tokenise means to convert the row text to some integers according to a vocabulary of possible elements
#In our case it means to create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) } #iterate over the character and create a lookup table (noting to do with stoi c function)
itos = { i:ch for i,ch in enumerate(chars) } #iterate over the character and create a lookup table

encode = lambda s: [stoi[c] for c in s]          # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) #decoder: take a list of integers, output a string

print(encode("ciao belli"))
print(decode(encode("ciao belli")))


[41, 47, 39, 53, 1, 40, 43, 50, 50, 47]
ciao belli


In [ ]:
#Let's now encode the entire text dataset and store it into a torch.Tensor
#A torch.Tensor is a multi-dimensional matrix containing elements of a single data type
import torch
data = torch.tensor(encode(text), dtype=torch.long) #A torch.dtype is an object that represents the data type of a torch.Tensor

print(data.shape,data.dtype)
print(data[:1000]) #the 1000 character we looked at earlier will to the GPT look like this


torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [ ]:
#Let's now split uo the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [ ]:
#We define a block size: the chunk of data that we randomly sample from the
#Train set to train the transformer
block_size = 8
train_data[:block_size+1]


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [ ]:
x = train_data[:block_size] # x are the input to the transformer. The first block_size characters
y = train_data[1:block_size+1] # y will the next block_size characters. It is offset by one because y is the target for each position in the input x
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [ ]:
#We define the batch: a group of many samble of size equal to block size
# stack in a single tensor. This is done for efficiencty to keeo the gpu busy
# and let it work on different samples of block size in parallel
torch.manual_seed(1337)
batch_size = 4 #how many indipendent sequences will we process in parallel (every forward/backward pass in the transformer)?
block_size = 8 #what is the maximum context lenght to make a prediction?

def get_batch(split):
  # generate a small batch of data of inputs x and targets y
  data = train_data if split == 'train' else val_data
  # Generating random positions from which take the chanck of data off
  #We generate batch_size numbers of random offset
  # In this case ix is gonna be 4 numbers that are randomly generated between zero and (len(data) - block_size)
  ix = torch.randint(len(data) - block_size, (batch_size,))
  #torch stack takes the random chosen 1-dimentional tensor and stack them togheter as rows of a 4 by 8 dimention tensor
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x,y

xb,yb = get_batch('train')
print('input:')
print(xb.shape)
print(xb)
print('target:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): #batch dimention
  for t in range(block_size): #time dimention
    context = xb[b,:t+1]
    target = yb[b, t]
    print(f"when input is {context.tolist()} the target: {target}")


input:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
target:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53, 

In [ ]:
 print(xb) #our input to the tranformer (used to feed the neural network)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [ ]:
#The simplest possible neural network that work good for LLM is the BigramLanguageModel
#Let's implement a pytorch module that implements the  BigramLanguageModel
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module): #BigramLanguageModel is a subclass of NN module

  def __init__(self,vocab_size):
    super().__init__()
    #Creating a token embedding table that is of size vocab_size x vocab_size
    #each token directly reads off logits for the next token from a lookup table
    self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)
    #nn.Embedding is a very thin wrapper around a tensor of shape vocab_size x vocab_size

  def forward(self, idx, targets=None): #target is optional becuse we use this function in situations were no loss is needed. By default is none
    #idx and targets are both (B,T) tensor of integers
    #when we pass idx to the token_embedding_table, every single integer in our input (idx) is
    # going to refer to the embedding table and is going to pick up a row corrisponding to that index
    # for example 24 will plcuk out the row 24 of the embedding table, and then 43, 58 etc
    # pytorch is going to arrange all this in a batch x time x channels tensor. In this case
    # is 4 x 8 x 65
    # logits are basically the scores for the next character in the sequence
    # we are predicting what's coming next based on echa individual token
    logits = self.token_embedding_table(idx) #(B,T,C)

    if targets is None: # in cases where we are interested just in the logits
      loss = None
    else:
      #loss function
      #A good way to measure a loss or like a quality of the poredictions
      #is to use the negative log likelihood loss which is implemented
      #implemented in pytorch under the name cross entropy
      # however the cross entropy function (see documentation) expexts logits and targets in the
      # form of (B*T,C) so we need to reshape our logits and targets
      B, T, C = logits.shape #Returns the size of the self tensor
      logits = logits.view(B*T, C) #Returns a new tensor with the same data as the self tensor but of a different shape
      targets = targets.view(B*T) # or targets.view(-1) This is a way of telling the library:
      #"give me a tensor that has these many columns and you compute the appropriate number of rows that is necessary to make this happen"
      loss = F.cross_entropy(logits, targets)
      # this measure the quality of the logits with respect to the targets
      # we can guess the loss that we expect is -ln(1/vocab_size) to be guessing good

    return logits, loss

  def generate(self, idx, max_new_tokens):
    # idx is (B,T) array of indeces in the current
    # this is the currenct context of some characters in some batch
    # the goal of the generate function is to generate (B,T+1), (B,T+2) etc..
    # so basically it continueus the generation in all the batch dimentions in the time dimention and it will do that for max_new_tokens
    for _ in range(max_new_tokens):
      #take the current indexes and get the predictions (logits) don't care about the loss
      logits, loss = self(idx) #this end up going to the forwared function above
      # focus only on the last time step (-1 is the last element in the time dimention)
      logits = logits[:,-1,:] #becomes (B,C)
      #apply softmax to get probabiloties
      probs = F.softmax(logits,dim=-1) # (B,C)
      #sample from the distribution of probabilities computed above and we ask pythorch to give us one sample
      idx_next = torch.multinomial(probs,num_samples=1) # index_next is of type (B,1) becuse for each batch we are going to have a single new prediction
      #append sampled index to the running sequence
      # whatever character is predicted, it is concatenated with the given context in the time dimention
      # that's become the next idx
      idx = torch.cat((idx,idx_next), dim=1) # (B,T+1)
    return idx

#Building and calling the BigramLanguageModel
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb,yb)
print(logits.shape)
print(loss)

#creating a 1 by 1 tensor and it's holidng a zero and the data type is integer
# so zero (new line) is were we are starting creating (the first character of the sequence we feed in)
idx = torch.zeros((1, 1), dtype=torch.long)
# starting generating from new line and asking for 100 tokens
# taking row number zero from the generated batch which will contains a series
# of indexes that will convert from a pytorch tensor into a python list so that we can feet it in the decode function to translate that into text
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

# we get garbage because the model is random and not trained
# also to make a prediction we only look at the last character of the context and we don't take into account the history

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [ ]:
#Let's train the model

#create a PyTorch optimizer. We are using AdamW (SGD is the basic optimizer, but Adamw is better expecially for small NN )
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
#lets create the optimizer object that will take the gradients and updated the parameters using the gradients
batch_size = 32

#tyical training loop
for step in range(10000):

  #sample a batch of data
  xb, yb = get_batch('train')

  #evaluate the loss
  logits, loss = m(xb, yb)
  #zeroising the gradients from the previous step
  optimizer.zero_grad(set_to_none=True)
  #getting the gradients for all the parameters
  loss.backward()
  #using those gradients to update our parameters
  optimizer.step()

print(loss.item())

2.4552481174468994


In [ ]:
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=500)[0].tolist()))

#This is a very basic model because the tokens are not talking to each other.
#We are only looking at the very last character to make a prediction of what comes next
#So now these tokens needs to talk to each other in order and figuring out what is in the context
#so that they can make better prediction of what comes next and that's why we are going to use a TRANSFORMER!


F anfRERFoksighiaimfrfreranthangur-
GOLLABAllile waricknthe y e pe s cke wbre cthe, thale las we, r,
O t
Tiea otry at aiey thivequrdrurencoue cy winchanot:
GHEO:
Butand, pe
I ine anelaiolfrd ther.
Angrthtr n t whe bofot lreaf hethilld brtou hance, ot beatous omyit gre orosathtis tt: th, rs ang my ourod;
Thesst
IOnd tingive she t, hang, WAUSO, hake ireys me s:
ink'le. ay, outcathathafomeat, Yons angr hor s pexand, h puce s changot,
MOMBu thage augd mou t o ngerksthe co s fr e XEYoouspeditheidrd's
